In [1]:
import numpy
print(numpy.__version__)


ModuleNotFoundError: No module named 'numpy'

In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt 
import gc
import os
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import sklearn
from sklearn.preprocessing import LabelEncoder
from xgboost.callback import EarlyStopping
from xgboost import XGBClassifier, callback
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

from lightgbm import early_stopping, log_evaluation, LGBMClassifier

import gc
# print('RAPIDS version',cudf.__version__)
import warnings
warnings.filterwarnings('ignore')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\Administrator\anaconda3\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Administrator\anaconda3\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Us

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\Administrator\anaconda3\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Administrator\anaconda3\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Us

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [ ]:
print(xgb.__version__)

In [ ]:
def optimize_types(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'object':
            num_unique = df[col].nunique()
            if num_unique / len(df[col]) < 0.5:
                df[col] = df[col].astype('category')
    return df

In [ ]:
import pandas as pd

# ▶️ 필요한 데이터만 불러오기
train = pd.read_csv('병합/98/train_final_2.csv')

# ▶️ 타깃/피처 분리
target_col = 'Segment'
feature_cols = [col for col in train.columns if col not in ['ID', target_col]]

X = train[feature_cols].copy()
y = train[target_col].copy()

In [ ]:
# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

# Categorical -> Numerical
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

encoders = {}  # 각 컬럼별 encoder 저장

for col in categorical_features:
    le = LabelEncoder()
    X.loc[:, col] = le.fit_transform(X[col])
    encoders[col] = le

In [ ]:
X = optimize_types(X)

print(X.info())

In [ ]:
# Unnamed가 있어 제거
X = X.drop(columns=["Unnamed: 0"])

In [ ]:
# Version 1
from collections import Counter

# 클래스별 샘플 수 확인
class_counts = Counter(y_encoded)

# 각 클래스의 비율에 맞춰 가중치 계산 (scale_pos_weight)
# 가중치는 각 클래스의 샘플 수의 비율에 반비례하는 값
total_samples = sum(class_counts.values())
weights = {cls: total_samples / count for cls, count in class_counts.items()}

# 특정 클래스에 대해 scale_pos_weight 설정
scale_pos_weight = weights[1]  # 예를 들어, 1번 클래스에 대해 가중치 적용

print("Calculated class weights:\n", weights)

# {3: 6.872025701376123, 4: 1.2486654887588888, 2: 18.810251587114976, 0: 2469.135802469136, 1: 16666.666666666668}

In [ ]:
gc.collect()

In [ ]:
# for XGB

X_sample = X.sample(n=100000, random_state=42)
y_sample = y_encoded[X_sample.index]

sample_weights = np.array([weights[class_label] for class_label in y_sample])

from xgboost import XGBClassifier

# GPU 안 쓰고 CPU로 안전하게 실행
temp_model = XGBClassifier(
    enable_categorical=True,
    verbosity=0,
    random_state=42
)

# 모델 학습
temp_model.fit(
    X_sample,
    y_sample,
    sample_weight=sample_weights
)


importance_dict = temp_model.get_booster().get_score(importance_type='gain')


if all(k in X.columns for k in importance_dict.keys()):
    importances = pd.Series(importance_dict)
else:
    # fallback: f0, f1 형식일 경우만 매핑
    feature_map = {f'f{idx}': col for idx, col in enumerate(X_sample.columns)}
    importances = pd.Series({feature_map.get(k, k): v for k, v in importance_dict.items()})

# 4. 메모리 정리
del temp_model
gc.collect()

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

importance_df = pd.DataFrame(importances.items(), columns=['feature', 'importance'])
importance_df = importance_df.sort_values(by='importance', ascending=False).reset_index(drop=True)

importance_df['cumsum'] = importance_df['importance'].cumsum()
importance_df['cumsum_ratio'] = importance_df['cumsum'] / importance_df['importance'].sum()
gc.collect()

In [ ]:
selected_features = importance_df[importance_df['cumsum_ratio'] <= 0.90]['feature'].tolist()
len(selected_features)

In [ ]:
X_reduced = X[selected_features]
gc.collect()

In [ ]:
del X_sample, y_sample, X, y, importances, importance_dict, importance_df
gc.collect()

In [ ]:
# 사용자 정의 평가 함수 (모델 학습엔 사용하지 않음)
def f1_eval_metric(preds, dtrain):
    labels = dtrain.get_label()
    preds_class = np.argmax(preds, axis=1)
    f1 = f1_score(labels, preds_class, average='macro')
    return 'f1_macro', f1

# Stratified K-Fold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# XGBoost 파라미터 (CPU 환경에 맞춤)
parameters = {
    'objective': 'multi:softprob',
    'num_class': len(np.unique(y_encoded)),
    'enable_categorical': True,
    'n_estimators': 10000,
    'random_state': 42,
    'n_jobs': -1,
    'use_label_encoder': False,
    'verbosity': 1,
    'tree_method': 'hist'  # GPU가 없으므로 hist로 변경
}

f1_scores = []
models = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_reduced, y_encoded)):
    print(f"\nFold {fold+1}/{n_splits}")
    
    X_train, X_val = X_reduced.iloc[train_idx], X_reduced.iloc[val_idx]
    y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]

    # 클래스별 샘플 가중치 적용
    sample_weights = np.array([weights[class_label] for class_label in y_train])

    model = XGBClassifier(**parameters)

    # 버전 최신화됐으므로 callback 사용 가능
    early_stop = callback.EarlyStopping(
        rounds=100,
        metric_name='mlogloss',  # f1_macro는 내장 아님
        data_name='validation_0',
        maximize=False,
        save_best=True
    )

    # 모델 학습
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        sample_weight=sample_weights,
        callbacks=[early_stop],
        verbose=100
    )

    # 예측 및 f1 평가
    preds_val = model.predict(X_val)
    f1 = f1_score(y_val, preds_val, average='macro')
    print(f"Fold {fold+1} F1 Score: {f1:.4f}")
    f1_scores.append(f1)
    models.append(model)  
    
    del model, X_train, X_val, y_train, y_val, preds_val
    gc.collect()

print("\nK-Fold F1 Scores:", np.round(f1_scores, 4))
print("평균 F1 Score:", np.mean(f1_scores))